# Chap03 - 텐서플로의 기본 이해하기

텐서플로의 핵심 구축 및 동작원리를 이해하고, 그래프를 만들고 관리하는 방법과 상수, 플레이스홀더, 변수 등 텐서플로의 '구성 요소'에 대해 알아보자.

## 3.1 연산 그래프

### 3.1.1 연산 그래프란?

그래프는 아래의 그림과 같이 노드(node)나 꼭지점(vertex)로 연결 되어 있는 개체(entity)의 집합을 부르는 용어다. 노드들은 변(edge)을 통해 서로 연결되어 있다. 

![](./images/graph.png)

데이터 흐름 그래프에서(DataFlow Grapy)의 변(edge) 어떤 노드에서 다른 노드로 흘러가는(flow) 데이터의 방향을 정한다.

텐서플로에서 그래프의 각 **노드는 하나의 연산을 나타내며, 입력값을 받아 다른 노드로 전달할 결과값을 출력**한다.

### 3.1.2 연산 그래프의 장점

텐서플로는 그래프의 연결 상태를 기반으로 연산을 최적화한다. 각 그래프에는 노드 간에 의존관계(dependency)가 존재한다.. 예를 들어, 아래의 그림 'A'에서 노드 `e`는 노드 `c`에 **직접의존**(direct dependeny)하고 있고, 노드 `a`에는 **간접의존**(indirect dependency) 한다.

![](./images/graph02.png)